## Generate samples for training / validation / testing

In [24]:
import os
import h5py
import numpy as np

len_seq = 36
n_bins = 96

days = ['2019'+ '%03d'%i for i in range(366)]
days += ['2020'+ '%03d'%i for i in range(366)]

data_path = '/home/yaoyi/lin00786/weather4cast/preprocess-data/R1'

all_samples = []
for day_in_year in days:

    cur_file = os.path.join(data_path, f'{day_in_year}.h5')
    if not os.path.exists(cur_file):
        continue
        
    hf = h5py.File(cur_file, 'r')
    blacklist = list(hf['data'].attrs['blacklist'])

    for i in range(n_bins - len_seq + 1):
            
        seq_bins = [i + j for j in range(len_seq)]
        bins_in_blacklist = [idx for idx in seq_bins if idx in blacklist]

        if len(bins_in_blacklist) == 0:
            all_samples.append([day_in_year, seq_bins[0], seq_bins[-1], None, None, None])
    
    if int(day_in_year) % 1000 == 365:
        next_day_in_year = '2020001'
    else:
        next_day_in_year = str(int(day_in_year) + 1)
                    
    next_file = os.path.join(data_path, f'{next_day_in_year}.h5')        
    if os.path.exists(next_file):
        
        next_hf = h5py.File(next_file, 'r')
        next_blacklist = list(next_hf['data'].attrs['blacklist'])
        
        for i in range(n_bins - len_seq + 1, n_bins):
            
            seq_bins = [i + j for j in range(len_seq) if i + j < n_bins]
            next_seq_bins = [(i + j) % n_bins for j in range(len_seq) if i + j >= n_bins]
            
            bins_in_blacklist = [idx for idx in seq_bins if idx in blacklist]
            bins_in_blacklist += [idx for idx in next_seq_bins if idx in next_blacklist]
                
            if len(bins_in_blacklist) == 0:
                all_samples.append([day_in_year, seq_bins[0], seq_bins[-1], next_day_in_year, next_seq_bins[0], next_seq_bins[-1]])
    

In [41]:
import csv

f = open('./samples.csv', 'w')
writer = csv.writer(f)
writer.writerow(['day_in_year', 'start_bin_id', 'end_bin_id', 'next_day_in_year', 'next_start_bin_id', 'next_end_bin_id'])

for i in all_samples:
    writer.writerow(list(i))

f.close()


In [42]:
import pandas as pd

pd.read_csv('./samples.csv', header=0)

,day_in_year,start_bin_id,end_bin_id,next_day_in_year,next_start_bin_id,next_end_bin_id
0,2019046,0,35,NaN,NaN,NaN
1,2019046,1,36,NaN,NaN,NaN
2,2019046,2,37,NaN,NaN,NaN
3,2019046,3,38,NaN,NaN,NaN
4,2019046,4,39,NaN,NaN,NaN
...,...,...,...,...,...,...
27868,2020045,56,91,NaN,NaN,NaN
27869,2020045,57,92,NaN,NaN,NaN
27870,2020045,58,93,NaN,NaN,NaN
27871,2020045,59,94,NaN,NaN,NaN
